In [7]:
import torch
import datetime
from torch.autograd import Variable
from torchvision.models import vgg
from torchvision.datasets import coco
from torchvision import transforms

In [13]:
BASE = '/Users/ajitpunj/Documents/School/cs 349d/project/'
m = vgg.vgg11(pretrained=True)

In [14]:
ANN_PATH = 'coco/cocoapi/annotations/instances_val2017.json'
VAL_PATH = 'coco/cocoapi/images/val2017/'
test_transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.CenterCrop(224), #also uses 224 cropped images
    transforms.ToTensor(),
])
def merge_coco(l):
    return torch.stack([x[0] for x in l])
c_dataset = coco.CocoDetection(root=BASE+VAL_PATH, annFile=BASE+ANN_PATH, transform=test_transform, )
eval_loader = torch.utils.data.DataLoader(c_dataset, batch_size=1, shuffle=True, collate_fn=merge_coco)

loading annotations into memory...
Done (t=1.36s)
creating index...
index created!


In [16]:
def compute_mem_size(t):
    t_size = t.size()
    n_nums = 1
    for n in t_size:
        n_nums *= n
    bit_size_map = {torch.float32 : 32, torch.float16 : 16, torch.float64 : 64, torch.uint8 : 8, torch.int8 : 8, torch.int16 : 16, torch.int32 : 32, torch.int64 : 64}
    return (n_nums * bit_size_map[t.dtype]) / 8

def vgg_measure_forward(model, x):
    print('Image size:', compute_mem_size(x))

    for module in model.classifier:
        t1 = datetime.datetime.now()
        x = module(x)
        t2 = datetime.datetime.now()
        dt1 = t2 - t1
        canon_name = type(module).__name__
        print('After ' + canon_name + '(' + str(dt1.total_seconds()) + '):  ' + str(compute_mem_size(x)))
           
    return x

In [17]:
m.eval()
for batch in eval_loader:
    vgg_measure_forward(m, batch)
    break

Image size: 602112.0


RuntimeError: size mismatch, m1: [672 x 224], m2: [25088 x 4096] at /Users/soumith/miniconda2/conda-bld/pytorch_1532621707135/work/aten/src/TH/generic/THTensorMath.cpp:2070

In [ ]:
transforms.ToPILImage()(sample_input)